In [5]:
# Importing Libraries

In [6]:
import pandas as pd
import numpy as np

In [7]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [8]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [9]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [10]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [11]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [12]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [13]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [14]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [15]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [30]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.normalization import BatchNormalization

In [17]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [18]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [19]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\visharag\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [20]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [21]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [23]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.cast instead.
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 26s 3ms/step - loss: 1.3306 - acc: 0.4361 - val_loss: 1.1743 - val_acc: 0.4723
Epoch 2/30
7352/7352 [==============================] - 24s 3ms/step - loss: 0.9716 - acc: 0.5788 - val_loss: 0.9656 - val_acc: 0.5263ETA: 0s - loss: 0.9773 -
Epoch 3/30
7352/7352 [==============================] - 25s 3ms/step - loss: 0.7787 - acc: 0.6510 - val_loss: 0.7841 - val_acc: 0.6135
Epoch 4/30
7352/7352 [==============================] - 26s 4ms/step - loss: 0.6911 - acc: 0.6587 - val_loss: 0.7109 - val_acc: 0.6203
Epoch 5/30
7352/7352 [==============================] - 25s 3ms/step - loss: 0.6496 - acc: 0.6794 - val_loss: 0.8000 - val_acc: 0.636200 - acc:
Epoch 6/30
7352/7352 [==============================] - 25s 3ms/step - loss: 0.8295 - acc: 0.6221 - val_loss: 0.7970 - val_acc: 0.6318 - ETA: 0s - loss: 0.8318 - ac
Epoch 7/30
7352/7352 [=========

# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

In [26]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 323us/step


In [27]:
score

[0.4692274864947224, 0.8805564981336953]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

# Changing dropout rate to 0.4

In [22]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [23]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.2))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [25]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 69s 9ms/step - loss: 1.1882 - acc: 0.5069 - val_loss: 1.0581 - val_acc: 0.5002
Epoch 2/30
7352/7352 [==============================] - 67s 9ms/step - loss: 0.8497 - acc: 0.6107 - val_loss: 0.7914 - val_acc: 0.6047
Epoch 3/30
7352/7352 [==============================] - 67s 9ms/step - loss: 0.7486 - acc: 0.6391 - val_loss: 0.8275 - val_acc: 0.5711
Epoch 4/30
7352/7352 [==============================] - 66s 9ms/step - loss: 0.7175 - acc: 0.6442 - val_loss: 0.7364 - val_acc: 0.6193
Epoch 5/30
7352/7352 [==============================] - 65s 9ms/step - loss: 0.6340 - acc: 0.7125 - val_loss: 0.6244 - val_acc: 0.7319
Epoch 6/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.5213 - acc: 0.7779 - val_loss: 0.5573 - val_acc: 0.7604
Epoch 7/30
7352/7352 [==============================] - 62s 8ms/step - loss: 0.4497 - acc: 0.8187 - val_loss: 0.5333 - val_acc: 0.7788
Epoch 8

In [26]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  0      420        67        3                   0   
STANDING                 0      122       406        4                   0   
WALKING                  0        0         0      452                  21   
WALKING_DOWNSTAIRS       0        0         0        1                 415   
WALKING_UPSTAIRS         0        6         0       10                   1   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                           23  
WALKING_DOWNSTAIRS                 4  
WALKING_UPSTAIRS                 454  


In [27]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 805us/step


In [28]:
score

[0.3021738717609319, 0.9015948422124194]

# Adding Batch Normalization Layer

In [28]:
# Initializing parameters
epochs = 30
batch_size = 32
n_hidden = 128

> Defining the Architecture of LSTM


In [31]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(BatchNormalization())
# Adding a dropout layer
model.add(Dropout(0.25))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               70656     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 774       
Total params: 71,942
Trainable params: 71,686
Non-trainable params: 256
_________________________________________________________________


In [32]:

# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [33]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 38s 5ms/step - loss: 1.0260 - acc: 0.5520 - val_loss: 0.9643 - val_acc: 0.6026
Epoch 2/30
7352/7352 [==============================] - 37s 5ms/step - loss: 0.7538 - acc: 0.6239 - val_loss: 0.8598 - val_acc: 0.6132
Epoch 3/30
7352/7352 [==============================] - 37s 5ms/step - loss: 0.6972 - acc: 0.6555 - val_loss: 0.9462 - val_acc: 0.5928
Epoch 4/30
7352/7352 [==============================] - 37s 5ms/step - loss: 0.6210 - acc: 0.6868 - val_loss: 0.7735 - val_acc: 0.6291
Epoch 5/30
7352/7352 [==============================] - 38s 5ms/step - loss: 0.5964 - acc: 0.7216 - val_loss: 0.9046 - val_acc: 0.6373
Epoch 6/30
7352/7352 [==============================] - 38s 5ms/step - loss: 0.4726 - acc: 0.8069 - val_loss: 0.4613 - val_acc: 0.8402
Epoch 7/30
7352/7352 [==============================] - 39s 5ms/step - loss: 0.3056 - acc: 0.9003 - val_loss: 0.3935 - val_acc: 0.8602
Epoch 8

In [34]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 533        4         0        0                   0   
SITTING                  0      368       121        0                   0   
STANDING                 0       50       482        0                   0   
WALKING                  0        3         0      480                   9   
WALKING_DOWNSTAIRS       0        0         0        8                 412   
WALKING_UPSTAIRS         0        8         1       28                  12   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           0  
WALKING                            4  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 422  


In [35]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 3s 950us/step


In [36]:
score

[0.38777748171767623, 0.9151679674244995]

## With a simple LSTM architecture we got 93.08% accuracy and a loss of 0.23 at epoch number 27

# stacking 2 LSTM

In [37]:
epochs1 = 30
batch_size1= 32
n_hidden1 = 128
n_hidden2 =64
pv1 = 0.2
pv2 = 0.5

In [38]:
# Initiliazing the sequential model
model1 = Sequential()
# Configuring the parameters
model1.add(LSTM(n_hidden1, return_sequences=True, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model1.add(Dropout(pv1))

model1.add(LSTM(n_hidden2))
# Adding a dropout layer
model1.add(Dropout(pv2))
# Adding a dense output layer with sigmoid activation
model1.add(Dense(n_classes, activation='sigmoid'))
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128, 128)          70656     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 390       
Total params: 120,454
Trainable params: 120,454
Non-trainable params: 0
_________________________________________________________________


In [39]:
# Compiling the model
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [40]:
# Training the model
model1.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs1)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 68s 9ms/step - loss: 1.1499 - acc: 0.5094 - val_loss: 1.0208 - val_acc: 0.4808
Epoch 2/30
7352/7352 [==============================] - 67s 9ms/step - loss: 0.8676 - acc: 0.6094 - val_loss: 0.7817 - val_acc: 0.6379
Epoch 3/30
7352/7352 [==============================] - 69s 9ms/step - loss: 0.6877 - acc: 0.6926 - val_loss: 0.6396 - val_acc: 0.7194
Epoch 4/30
7352/7352 [==============================] - 70s 9ms/step - loss: 0.5113 - acc: 0.7790 - val_loss: 0.4615 - val_acc: 0.8069
Epoch 5/30
7352/7352 [==============================] - 66s 9ms/step - loss: 0.3618 - acc: 0.8731 - val_loss: 0.3981 - val_acc: 0.8521
Epoch 6/30
7352/7352 [==============================] - 68s 9ms/step - loss: 0.2408 - acc: 0.9166 - val_loss: 0.3679 - val_acc: 0.8802
Epoch 7/30
7352/7352 [==============================] - 66s 9ms/step - loss: 0.2124 - acc: 0.9279 - val_loss: 0.3804 - val_acc: 0.8789
Epoch 8

In [41]:
score1 = model1.evaluate(X_test, Y_test)

2947/2947 [==============================] - 5s 2ms/step


In [42]:
score1

[0.5845529814390626, 0.8880217170003394]

## With a stacked LSTM architecture we got 92.13% accuracy and a loss of 0.36 at epoch number 23

# I got accuracy of 93.08% in previous simple LSTM layer..